In [1]:
import pickle

# Load encoders
with open('label_encoders.pkl', 'rb') as f:
    encoders = pickle.load(f)

# Print mappings
for col, le in encoders.items():
    mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    print(f"Column: {col}")
    print(mapping)
    print("-" * 30)


FileNotFoundError: [Errno 2] No such file or directory: 'label_encoders.pkl'